<a href="https://colab.research.google.com/github/skywalkerCam/Analisando-Sentimentos/blob/main/Oficina09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação das Bibliotecas

**Bibliotecas necessárias:**

**pandas e numpy:** Usadas para manipulação de dados. O pandas ajuda na organização dos dados em tabelas (DataFrames), enquanto o numpy é usado para manipulação de arrays e operações numéricas.

**sklearn.model_selection.train_test_split:** Função que divide o conjunto de dados em dois subconjuntos: um para treinamento e outro para teste do modelo.

**sklearn.feature_extraction.text.TfidfVectorizer:** Usada para converter textos em uma forma numérica que o modelo pode entender. A técnica TF-IDF ajuda a calcular a relevância de uma palavra em um documento, levando em conta a frequência da palavra no texto e a raridade dela em um conjunto maior.

**sklearn.naive_bayes.MultinomialNB:** O modelo Naive Bayes, especificamente a versão Multinomial, é usado para classificação de texto, como no caso de categorizar textos em diferentes classes.

**sklearn.pipeline.make_pipeline:** Facilita a criação de um fluxo de trabalho, conectando várias etapas (como transformação de texto e treinamento de modelo) em uma única linha de código.

**sklearn.metrics.accuracy_score e classification_report:** Usados para avaliar a performance do modelo, calculando a precisão, recall, f1-score e outras métricas.

**também preparamos o NLTK (Natural Language Toolkit) para fazer pré-processamento de texto. As funções baixam recursos que serão úteis para manipular texto:**

**stopwords:** São palavras como "a", "de", "em", "o", que são muito comuns e geralmente não agregam valor ao entendimento do texto, sendo removidas na maioria dos casos.

**punkt:** Utilizado para dividir o texto em tokens (palavras ou frases).

**wordnet:** Um dicionário semântico usado para lemmatização, ou seja, reduzir palavras à sua forma base.

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download("punkt")
nltk.download("wordnet")
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Carregar o DataSet

In [ ]:
# Definir o caminho dos arquivos
amazon_file = "amazon_cells_labelled.txt"
imdb_file = "imdb_labelled.txt"
yelp_file = "yelp_labelled.txt"

# Criar uma função para carregar os arquivos
def load_sentiment_data(filepath):
    return pd.read_csv(filepath, delimiter="\t", header=None, names=["sentence", "label"])

# Carregar os três datasets
amazon_df = load_sentiment_data(amazon_file)
imdb_df = load_sentiment_data(imdb_file)
yelp_df = load_sentiment_data(yelp_file)

# Exibir as primeiras linhas dos datasets
print("Amazon Dataset:")
print(amazon_df.head())

print("\nIMDb Dataset:")
print(imdb_df.head())

print("\nYelp Dataset:")
print(yelp_df.head())


Amazon Dataset:
                                            sentence  label
0  So there is no way for me to plug it in here i...      0
1                        Good case, Excellent value.      1
2                             Great for the jawbone.      1
3  Tied to charger for conversations lasting more...      0
4                                  The mic is great.      1

IMDb Dataset:
                                            sentence  label
0  A very, very, very slow-moving, aimless movie ...      0
1  Not sure who was more lost - the flat characte...      0
2  Attempting artiness with black & white and cle...      0
3       Very little music or anything to speak of.        0
4  The best scene in the movie was when Gerardo i...      1

Yelp Dataset:
                                            sentence  label
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.  

Unindo os Dataset

In [ ]:
# Concatenar os datasets
df = pd.concat([amazon_df, imdb_df, yelp_df], ignore_index=True)

# Verificar o tamanho do dataset
print("Total de frases:", df.shape[0])


Total de frases: 2748


# Pré-Processamento

Antes de aplicar o modelo de aprendizado de máquina, precisamos transformar os textos brutos em uma forma que o modelo consiga processar. A técnica que vamos utilizar é o TfidfVectorizer, que transforma os textos em uma matriz de características numéricas, representando a importância das palavras nos textos.

O pré-processamento é fundamental para garantir que o modelo compreenda o texto de forma eficaz, removendo palavras irrelevantes (stopwords) e transformando palavras semelhantes em uma representação numérica. O TF-IDF (Term Frequency - Inverse Document Frequency) é uma técnica que ajuda a identificar palavras importantes, dando maior peso às palavras que são raras, mas significativas no contexto do documento.

In [ ]:

# Inicializar o lematizador
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    # Remover caracteres especiais e números
    text = re.sub(r"[^a-z\s]", "", text)
    # Tokenizar
    tokens = word_tokenize(text)
    # Remover stopwords e aplicar lematização
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Unir tokens limpos em um texto final
    return " ".join(processed_tokens)

    # Aplicar pré-processamento ao dataset
df["clean_text"] = df["sentence"].apply(preprocess_text)

# Exibir as primeiras linhas do dataset limpo
print(df.head())



                                            sentence  label  \
0  So there is no way for me to plug it in here i...      0   
1                        Good case, Excellent value.      1   
2                             Great for the jawbone.      1   
3  Tied to charger for conversations lasting more...      0   
4                                  The mic is great.      1   

                                          clean_text  
0                     way plug u unless go converter  
1                          good case excellent value  
2                                      great jawbone  
3  tied charger conversation lasting minutesmajor...  
4                                          mic great  


# Pipeline de Pré-processamento e Classificação
Utilizaremos um pipeline que inclui a transformação dos dados (com TfidfVectorizer) e o modelo de classificação (Naive Bayes).

# Modelo de Classificação

Vamos usar o modelo Naive Bayes para a análise de sentimentos. O Naive Bayes é um bom modelo para tarefas de classificação de texto, pois ele funciona bem com dados de alta dimensionalidade e é simples de treinar. Ele é particularmente eficaz para a análise de sentimentos, onde as palavras desempenham um papel importante na classificação.

O modelo é treinado com os dados de treinamento, que consistem em comentários rotulados como positivos ou negativos. O Naive Bayes aprende a partir da frequência das palavras e suas associações com as classes (0 ou 1).


In [ ]:
# Inicializar o TF-IDF Vectorizer
tfidf = TfidfVectorizer()

# Separar dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.2, random_state=42)

pipeline = make_pipeline(
    TfidfVectorizer(stop_words='english'),  # Transforma os textos em vetores TF-IDF
    MultinomialNB()  # Modelo Naive Bayes
)
# Treinar o modelo
pipeline.fit(X_train, y_train)

# Treinar o modelo
pipeline.fit(X_train, y_train)



Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(stop_words='english')),
                ('multinomialnb', MultinomialNB())])

# Avaliação do Modelo

vamos avaliar o modelo usando os dados de teste. A métrica principal que vamos usar é a acurácia, que nos diz a porcentagem de previsões corretas feitas pelo modelo.

A acurácia representa a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. No caso da análise de sentimentos, a acurácia indica quão bem o modelo classifica os comentários como positivos ou negativos.



In [ ]:
# Fazer previsões nos dados de teste
y_pred = pipeline.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy * 100:.2f}%')

# Relatório de classificação (precisão, recall e f1-score)
print(classification_report(y_test, y_pred))



Acurácia do modelo: 80.55%
              precision    recall  f1-score   support

           0       0.88      0.73      0.80       291
           1       0.75      0.89      0.81       259

    accuracy                           0.81       550
   macro avg       0.81      0.81      0.81       550
weighted avg       0.82      0.81      0.80       550



# Análise do Desempenho

Podemos observar que o modelo acerta aproximadamente 81% das previsões, um valor satisfatório, considerando que modelos de NLP geralmente têm acurácia entre 75% e 85%.

# Análise por Classes

**Para a precisão(Precision), temos que:**

Para classe 0 (negativo): 88% das previsões de "negativo" estavam corretas.
Para classe 1 (positivo): 75% das previsões de "positivo" estavam corretas.

**Para a Revocação (Recall), temos que:**

O modelo identificou 73% dos comentários negativos corretamente.
Ele identificou 89% dos comentários positivos corretamente.

Isso indica que o modelo tem maior facilidade para encontrar comentários positivos (alta revocação para 1), mas pode estar perdendo algumas ocorrências de negativos.

**Para a F1-Score, temos que:**

O F1-Score equilibra precisão e revocação.
Ambas as classes têm um F1-score próximo de 80%, indicando um bom equilíbrio.
